<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 4
<br>
<b>Deadline:</b> March 30, 2020 (Monday). 23:00
</div>

# Exercise 5. Sequence-to-sequence modeling with recurrent neural networks

The goals of this exercise are
* to get familiar with recurrent neural networks used for sequential data processing
* to get familiar with the sequence-to-sequence model for machine translation
* to learn PyTorch tools for batch processing of sequences with varying lengths
* to learn how to write a custom `DataLoader`

You may find it useful to look at this tutorial:
* [Translation with a Sequence to Sequence Network and Attention](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [1]:
skip_training = True# Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [3]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import tools
import tests

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Data

The dataset that we are going to use consists of pairs of sentences in French and English.

In [7]:
from data import TranslationDataset, MAX_LENGTH, SOS_token, EOS_token

trainset = TranslationDataset(data_dir, train=True)

* `TranslationDataset` supports indexing as required by `torch.utils.data.Dataset`.
* Sentences are tensors of maximum length `MAX_LENGTH`.
* Words in a (sentence) tensor are represented as an index (integer) in a language vocabulary.
* The string representation of a word from the source language can be obtained from index `i` with `dataset.input_lang.index2word[i]`.
* Similarly for the target language `dataset.output_lang.index2word[j]`.

Let us look at samples from that dataset.

In [8]:
src_sentence, tgt_sentence = trainset[np.random.choice(len(trainset))]
print('Source sentence: "%s"' % ' '.join(trainset.input_lang.index2word[i.item()] for i in src_sentence))
print('Sentence as tensor of word indices:')
print(src_sentence)

print('Target sentence: "%s"' % ' '.join(trainset.output_lang.index2word[i.item()] for i in tgt_sentence))
print('Sentence as tensor of word indices:')
print(tgt_sentence)

Source sentence: "c est que nous appelons un pionnier . EOS"
Sentence as tensor of word indices:
tensor([ 145,   25,  914,  123, 3629,   66, 3628,    5,    1])
Target sentence: "he is what we call a pioneer . EOS"
Sentence as tensor of word indices:
tensor([  14,   40, 1471,   77, 1389,   42, 2266,    4,    1])


In [9]:
print('Number of source-target pairs in the training set: ', len(trainset))

Number of source-target pairs in the training set:  8682


## Sequence-to-sequence model for machine translation

In this exercise, we are going to build a machine translation system which transforms a sentence in one language into a sentence in another one. The computational graph of the translation model is shown below:

<img src="seq2seq.png" width=900>

We are going to use a simplified model without the dotted connections.

## Custom DataLoader

We would like to train the sequence-to-sequence model using mini-batch training.
One difficulty of mini-batch training in this case is that sequences may have varying lengths and this has to be taken into account when building the computational graph. Luckily, PyTorch has tools to support batch processing of such sequences.
To use those tools, we need to write a custom data loader which puts sequences of varying lengths in the same tensor. We can customize the data loader by providing a custom `collate_fn` as explained [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

Our collate function:
- combines sequences from the source language in a single tensor with extra values (at the end) filled with `padding_value=0`.
- combines sequences from the target language in a single tensor with extra values (at the end) filled with `padding_value=0`.

**Important**:
- Late in the code (not in this `collate` function), we will convert source sequences to objects of class [`PackedSequence`](https://pytorch.org/docs/stable/nn.html?highlight=packedsequence#torch.nn.utils.rnn.PackedSequence) which can be processed by recurrent units such as `GRU` or `LSTM`. `PackedSequence` requires sequences to be sorted by their lengths.
**Therefore, the returned source sequences should be sorted by length in a decreasing order.**
* The target sequences need not be sorted by their lengths because we have to keep the same order of sequences in the source and target tensors.

Your task is to implement the collate function.

In [10]:
padding_value = 0

In [11]:
from torch.nn.utils.rnn import pad_sequence

def collate(list_of_samples):
    """Merges a list of samples to form a mini-batch.

    Args:
      list_of_samples is a list of tuples (src_seq, tgt_seq):
          src_seq is of shape (src_seq_length,)
          tgt_seq is of shape (tgt_seq_length,)

    Returns:
      src_seqs of shape (max_src_seq_length, batch_size): Tensor of padded source sequences.
          The sequences should be sorted by length in a decreasing order, that is src_seqs[:,0] should be
          the longest sequence, and src_seqs[:,-1] should be the shortest.
      src_seq_lengths: List of lengths of source sequences.
      tgt_seqs of shape (max_tgt_seq_length, batch_size): Tensor of padded target sequences.
    """
    # YOUR CODE HERE
    try:
        list_of_samples = sorted(list_of_samples, key=lambda x: len(x[0]), reverse=True)
        src_seqs, tgt_seqs = zip(*list_of_samples)
        src_seq_length = [len(x) for x in src_seqs]
        
        src_seqs = pad_sequence(src_seqs, padding_value=padding_value)
        tgt_seqs = pad_sequence(tgt_seqs, padding_value=padding_value)
        return src_seqs, src_seq_length, tgt_seqs
    except:
        raise NotImplementedError()

In [12]:
def test_collate_shapes():
    pairs = [
        (torch.LongTensor([1, 2]), torch.LongTensor([3, 4, 5])),
        (torch.LongTensor([6, 7, 8]), torch.LongTensor([9, 10])),
    ]
    pad_src_seqs, src_seq_lengths, pad_tgt_seqs = collate(pairs)
    assert pad_src_seqs.shape == torch.Size([3, 2]), f"Bad pad_src_seqs.shape: {pad_src_seqs.shape}"
    assert pad_src_seqs.dtype == torch.long
    assert pad_tgt_seqs.shape == torch.Size([3, 2]), f"Bad pad_tgt_seqs.shape: {pad_tgt_seqs.shape}"
    assert pad_tgt_seqs.dtype == torch.long
    print('Success')

test_collate_shapes()

Success


In [13]:
# This cell tests collate() function

In [14]:
# We create custom DataLoader using the implemented collate function
# We are going to process 64 sequences at the same time (batch_size=64)
from torch.utils.data import DataLoader
trainloader = DataLoader(dataset=trainset, batch_size=64, shuffle=True, collate_fn=collate, pin_memory=True)

## Encoder

The encoder encodes a source sequence $(x_1, x_2, ..., x_T)$ into a single vector $h_T$ using the following recursion:
$$
  h_{t} = f(h_{t-1}, x_t) \qquad t = 1, \ldots, T
$$
where:
* intial state $h_0$ is often chosen arbitrarily (we choose it to be zero)
* function $f$ is defined by the type of the RNN cell (in our experiments, we will use [GRU](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU))
* $x_t$ is a vector that represents the $t$-th word in the source sentence.

A common practice in natural language processing is to _learn_ the word representations $x_t$ (instead of, for example, using one-hot coded vectors). In PyTorch, this is supported by class [Embedding](https://pytorch.org/docs/stable/nn.html#torch.nn.Embedding) which we are going to use.

The computational graph of the encoder is shown below:

<img src="seq2seq_encoder.png" width=500>

Your task is to implement the `forward` function of the encoder. It should contain the following steps:
* Embed the words of the source sequences.
* Pack source sequences using [`pack_padded_sequence`](https://pytorch.org/docs/stable/nn.html?highlight=pack_padded_sequence#torch.nn.utils.rnn.pack_padded_sequence). This converts padded source sequences into an object that can be processed by PyTorch recurrent units such as `nn.GRU` or `nn.LSTM`.
* Apply GRU computations to packed sequences obtained in the previous step
* Convert packed sequence of GRU outputs into padded representation with [`pad_packed_sequence`](https://pytorch.org/docs/stable/nn.html?highlight=pad_packed_sequence#torch.nn.utils.rnn.pad_packed_sequence).

In [15]:
class Encoder(nn.Module):
    def __init__(self, src_dictionary_size, embed_size, hidden_size):
        """
        Args:
          src_dictionary_size: The number of words in the source dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state of GRU.
        """
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(src_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)

    def forward(self, pad_seqs, seq_lengths, hidden):
        """
        Args:
          pad_seqs of shape (max_seq_length, batch_size): Padded source sequences.
          seq_lengths: List of sequence lengths.
          hidden of shape (1, batch_size, hidden_size): Initial states of the GRU.

        Returns:
          outputs of shape (max_seq_length, batch_size, hidden_size): Padded outputs of GRU at every step.
          hidden of shape (1, batch_size, hidden_size): Updated states of the GRU.
        """
        # YOUR CODE HERE
        try:
            embedding = self.embedding(pad_seqs)
            packed = pack_padded_sequence(embedding, seq_lengths)
            print(packed)
            outputs, hidden = self.gru(packed, hidden)
            outputs, _ = pad_packed_sequence(outputs)
            return outputs, hidden
        except:
            raise NotImplementedError()
    
    def init_hidden(self, batch_size=1):
        return torch.zeros(1, batch_size, self.hidden_size)

In [16]:
def test_Encoder_shapes():
    hidden_size = 3
    encoder = Encoder(src_dictionary_size=5, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    hidden = encoder.init_hidden(batch_size=batch_size)
    pad_seqs = torch.tensor([
        [1, 2],
        [2, 3],
        [3, 0],
        [4, 0]
    ])

    outputs, new_hidden = encoder.forward(pad_seqs=pad_seqs, seq_lengths=[4, 2], hidden=hidden)
    assert outputs.shape == torch.Size([4, batch_size, hidden_size]), f"Bad outputs.shape: {outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape: {new_hidden.shape}"
    print('Success')

test_Encoder_shapes()

PackedSequence(data=tensor([[-1.2423, -0.0124,  0.5017,  0.8370, -1.4228, -0.2938,  2.0453, -0.3844,
          0.8609,  0.9017],
        [-0.6772,  2.3649, -0.5501, -2.3647, -1.0711,  1.6568, -0.9419,  0.9434,
          0.3803,  1.3666],
        [-0.6772,  2.3649, -0.5501, -2.3647, -1.0711,  1.6568, -0.9419,  0.9434,
          0.3803,  1.3666],
        [ 1.0180, -1.1960, -0.0071,  0.1918, -2.6715,  0.9023, -0.0368,  1.5802,
          0.1921, -0.6606],
        [ 1.0180, -1.1960, -0.0071,  0.1918, -2.6715,  0.9023, -0.0368,  1.5802,
          0.1921, -0.6606],
        [-1.9189, -1.3079,  1.8937, -0.8635,  0.3058,  0.5225, -0.5711, -0.3023,
         -0.8606,  0.0335]], grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([2, 2, 1, 1]), sorted_indices=None, unsorted_indices=None)
Success


In [17]:
tests.test_Encoder(Encoder)

PackedSequence(data=tensor([[-0.5000,  0.2000],
        [ 0.1000, -0.4000],
        [ 0.1000, -0.4000],
        [-0.3000,  0.4000],
        [-0.3000,  0.4000],
        [-0.1000,  0.8000]]), batch_sizes=tensor([2, 2, 1, 1]), sorted_indices=None, unsorted_indices=None)
outputs[:, 0, :]:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0221],
        [ 0.0007, -0.0055],
        [ 0.0005,  0.0323]])
expected:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0221],
        [ 0.0007, -0.0055],
        [ 0.0005,  0.0323]])
outputs[:2, 1, :]:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0021]])
expected:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0021]])
new_hidden:
 tensor([[[ 0.0005,  0.0323],
         [ 0.0004, -0.0021]]])
expected:
 tensor([[[ 0.0005,  0.0323],
         [ 0.0004, -0.0021]]])
Success


## Decoder

The decoder takes as input the representation computed by the encoder and transforms it into a sentence in the target language. The computational graph of the decoder is shown below:

<img src="seq2seq_decoder.png" width=500 align="top">

* $z_0$ is the output of the encoder, that is $z_0 = h_5$, thus `hidden_size` of the decoder should be the same as `hidden_size` of the encoder.
* $y_{i}$ are the log-probabilities of the words in the target language, the dimensionality of $y_{i}$ is the size of the target dictionary.
* $z_{i}$ is mapped to $y_{i}$ using a linear layer `self.out` followed by `F.log_softmax` (because we use `nn.NLLLoss` loss for training).
* Each cell of the decoder is a GRU, it receives as inputs the previous state $z_{i-1}$ and relu of the **embedding** of the previous word. Thus, you need to embed the words of the target language as well. The previous word is taken as the word with the maximum log-probability.

Note that the decoder outputs a word at every step and the same word is used as the input to the recurrent unit at the next step. At the beginning of decoding, the previous word input is fed with a special word SOS which stands for "start of a sentence". During training, we know the target sentence for decoding, therefore we can feed the correct words $y_i$ as inputs to the recurrent unit.

There is one extra thing that it is wise to take care of. When the target sentence is fed to the decoder during training, the decoder learns to generate only the next word (this scenario is called "teacher forcing"). At test time, the decoder works differently: it generates the whole sequence using its own predictions as inputs at each step. Therefore, it makes sense to train the decoder to produce full sentences. In order to do that, we will alternate between two modes during training:
* "teacher forcing": the decoder is fed with the words in the target sequence
* no "teacher forcing": the decoder generates the output sequence using its own predictions. We will limit the maximum length of generated sequences to `MAX_LENGTH`.

You need to implement the decoder which has the structure shown in the figure above.

In [18]:
class Decoder(nn.Module):
    def __init__(self, tgt_dictionary_size, embed_size, hidden_size):
        """
        Args:
          tgt_dictionary_size: The number of words in the target dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state.
        """
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(tgt_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)
        self.out = nn.Linear(hidden_size, tgt_dictionary_size)

    def forward(self, hidden, pad_tgt_seqs=None, teacher_forcing=False):
        """
        Args:
          hidden of shape (1, batch_size, hidden_size): States of the GRU.
          pad_tgt_seqs of shape (max_out_seq_length, batch_size): Tensor of words (word indices) of the
              target sentence. If None, the output sequence is generated by feeding the decoder's outputs
              (teacher_forcing has to be False).
          teacher_forcing (bool): Whether to use teacher forcing or not.

        Returns:
          outputs of shape (max_out_seq_length, batch_size, tgt_dictionary_size): Tensor of log-probabilities
              of words in the target language.
          hidden of shape (1, batch_size, hidden_size): New states of the GRU.

        Note: Do not forget to transfer tensors that you may want to create in this function to the device
        specified by `hidden.device`.
        """
        if pad_tgt_seqs is None:
            assert not teacher_forcing, 'Cannot use teacher forcing without a target sequence.'

        # YOUR CODE HERE
        try:
            out_length = pad_tgt_seqs.shape[0] if pad_tgt_seqs is not None else MAX_LENGTH 
            input_ = torch.tensor(SOS_token * np.ones((1, hidden.size(1))), device=hidden.device, dtype=torch.int64)
            outputs = []
            for i in range(out_length):
                embedding = self.embedding(input_.view(1,-1))
                output, hidden = self.gru(F.relu(embedding), hidden)
                output = self.out(output)
                output = F.log_softmax(output, dim=2)
                outputs.append(output)
                if teacher_forcing:
                    input_ = pad_tgt_seqs[i]
                else:
                    input_ = torch.max(output,dim=2)[1].detach()
            outputs = torch.cat(outputs, dim=0)
            return outputs, hidden
        except:
            raise NotImplementedError()

In [19]:
def test_Decoder_shapes():
    hidden_size = 2
    tgt_dictionary_size = 5
    test_decoder = Decoder(tgt_dictionary_size, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    pad_tgt_seqs = torch.tensor([
        [1, 2],
        [2, 3],
        [3, 0],
        [4, 0]
    ])  # [max_seq_length, batch_size]

    hidden = torch.zeros(1, batch_size, hidden_size)
    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=False)

    assert outputs.size(0) <= 4, f"Too long output sequence: outputs.size(0)={outputs.size(0)}"
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=True)
    assert outputs.shape == torch.Size([4, batch_size, tgt_dictionary_size]), \
        f"Bad shape outputs.shape={outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    # Generation mode
    outputs, new_hidden = test_decoder.forward(hidden, None, teacher_forcing=False)
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    print('Success')

test_Decoder_shapes()

Success


In [20]:
tests.test_Decoder_no_forcing(Decoder)
tests.test_Decoder_with_forcing(Decoder)
tests.test_Decoder_generation(Decoder)

outputs[:, 0, :]:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
expected:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
outputs[:, 1, :]:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
expected:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
new_hidden:
 tensor([[[0.1003, 0.0421],
         [0.1003

## Training of sequence-to-sequence model using mini-batches

Now we are going to train the sequence-to-sequence model on the toy translation dataset.

In [21]:
# Create the seq2seq model
hidden_size = embed_size = 256
encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size).to(device)
decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size).to(device)

In [22]:
teacher_forcing_ratio = 0.5

Implement the training loop in the cell below. In the training loop, we first encode source sequences using the encoder, then we decode the encoded state using the decoder. The decoder outputs log-probabilities of words in the target language. We need to use these log-probabilities and the indexes of the words in the target sequences to compute the loss.

Recommended hyperparameters:
- Encoder optimizer: Adam with learning rate 0.001
- Decoder optimizer: Adam with learning rate 0.001
- Number of epochs: 30
- Toggle `teacher_forcing` on and off (for each mini-batch) according to the `teacher_forcing_ratio` specified above.

Hints:
- Training should proceed relatively fast.
- If you do well, the training loss should reach 0.1 in 30 epochs.
- **Important:** When computing the loss, you need to ignore the padded values. This can easily be done by using argument `ignore_index` of function [`nll_loss`](
https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.nll_loss).

In [23]:
if not skip_training:
    # YOUR CODE HERE
    try:
        import random
        ENoptimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)
        DEoptimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)
        
        for epoch in range(30):
            for i, batch in enumerate(trainloader):
                ENoptimizer.zero_grad()
                DEoptimizer.zero_grad()
                
                input_seq, input_seq_len, target_seq = batch 
                input_seq, target_seq = input_seq.to(device), target_seq.to(device)
                batch_size = input_seq.shape[1]
                
                ENhidden = encoder.init_hidden(batch_size).to(device)
                _, ENhidden = encoder(input_seq, input_seq_len, ENhidden)
                teacher_forcing = True if random.uniform(0,1) < teacher_forcing_ratio else False
                DEoutputs, DEhidden = decoder(ENhidden, target_seq, teacher_forcing=teacher_forcing)
                
                loss = 0
                for i in range(DEoutputs.shape[0]):
                    loss = loss + F.nll_loss(DEoutputs[i], target_seq[i], ignore_index=0) 
                loss = loss/len(DEoutputs[0])
                loss.backward()
                
                ENoptimizer.step()
                DEoptimizer.step()
    except:
        raise NotImplementedError()

PackedSequence(data=tensor([[ 0.8802, -0.6379,  0.6623,  ...,  0.2814, -1.4391, -0.9124],
        [ 0.4771,  1.7098, -0.1462,  ...,  1.3521, -0.0677,  0.5578],
        [ 0.8802, -0.6379,  0.6623,  ...,  0.2814, -1.4391, -0.9124],
        ...,
        [-1.5637,  0.1523,  1.4797,  ..., -0.8766, -0.2085, -0.3420],
        [-1.5637,  0.1523,  1.4797,  ..., -0.8766, -0.2085, -0.3420],
        [-1.5637,  0.1523,  1.4797,  ..., -0.8766, -0.2085, -0.3420]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 64, 55, 37, 19, 13, 11]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.7000, -1.0763, -1.8274,  ...,  0.1813, -0.2524, -1.3005],
        [ 0.7000, -1.0763, -1.8274,  ...,  0.1813, -0.2524, -1.3005],
        [-0.5359,  0.6534, -0.5080,  ...,  0.0445,  0.1146, -0.3603],
        ...,
        [-1.5647,  0.1533,  1.4787,  ..., -0.8776, -0.2095, -0.3410],
        [-1.5647,  0.1533,  1.4787,  ..., -0.8776, -0.2095, -0.3410],
        [-1

PackedSequence(data=tensor([[ 0.8671, -0.6260,  0.6570,  ...,  0.2710, -1.4527, -0.8993],
        [ 0.8671, -0.6260,  0.6570,  ...,  0.2710, -1.4527, -0.8993],
        [ 0.9411,  0.7290,  0.4508,  ...,  1.7479,  0.1384,  1.2227],
        ...,
        [-1.5769,  0.1645,  1.4901,  ..., -0.8896, -0.2214, -0.3320],
        [-1.5769,  0.1645,  1.4901,  ..., -0.8896, -0.2214, -0.3320],
        [-1.5769,  0.1645,  1.4901,  ..., -0.8896, -0.2214, -0.3320]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 63, 56, 38, 24, 16,  4]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.9404,  0.7290,  0.4516,  ...,  1.7471,  0.1383,  1.2227],
        [ 0.8661, -0.6251,  0.6577,  ...,  0.2700, -1.4537, -0.8984],
        [ 0.8661, -0.6251,  0.6577,  ...,  0.2700, -1.4537, -0.8984],
        ...,
        [-1.5777,  0.1652,  1.4909,  ..., -0.8903, -0.2221, -0.3317],
        [-1.5777,  0.1652,  1.4909,  ..., -0.8903, -0.2221, -0.3317],
        [-1

PackedSequence(data=tensor([[ 0.5033,  1.2756, -1.1483,  ...,  1.1805,  2.6638,  0.7072],
        [ 0.8536, -0.6122,  0.6702,  ...,  0.2567, -1.4679, -0.8930],
        [ 0.8536, -0.6122,  0.6702,  ...,  0.2567, -1.4679, -0.8930],
        ...,
        [-1.5831,  0.1686,  1.4968,  ..., -0.8954, -0.2273, -0.3302],
        [-1.5831,  0.1686,  1.4968,  ..., -0.8954, -0.2273, -0.3302],
        [-1.5831,  0.1686,  1.4968,  ..., -0.8954, -0.2273, -0.3302]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 63, 63, 54, 37, 25, 13,  6]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.8526, -0.6110,  0.6714,  ...,  0.2558, -1.4692, -0.8929],
        [ 0.5036,  1.2748, -1.1491,  ...,  1.1813,  2.6647,  0.7068],
        [ 0.8526, -0.6110,  0.6714,  ...,  0.2558, -1.4692, -0.8929],
        ...,
        [-1.5833,  0.1684,  1.4970,  ..., -0.8955, -0.2276, -0.3302],
        [-1.5833,  0.1684,  1.4970,  ..., -0.8955, -0.2276, -0.3302],
        [-1

PackedSequence(data=tensor([[ 0.4048,  0.0760, -1.2922,  ...,  0.2585, -1.6433,  0.5268],
        [ 0.8428, -0.5969,  0.6875,  ...,  0.2422, -1.4860, -0.8918],
        [ 0.9244,  0.7402,  0.4672,  ...,  1.7428,  0.1339,  1.2351],
        ...,
        [-1.5841,  0.1646,  1.4986,  ..., -0.8959, -0.2303, -0.3306],
        [-1.5841,  0.1646,  1.4986,  ..., -0.8959, -0.2303, -0.3306],
        [-1.5841,  0.1646,  1.4986,  ..., -0.8959, -0.2303, -0.3306]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 64, 51, 43, 29, 17,  6]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.7213, -1.0684, -1.7859,  ...,  0.1488, -0.2407, -1.2897],
        [ 0.8428, -0.5963,  0.6885,  ...,  0.2414, -1.4873, -0.8915],
        [ 0.8428, -0.5963,  0.6885,  ...,  0.2414, -1.4873, -0.8915],
        ...,
        [-1.5841,  0.1644,  1.4987,  ..., -0.8959, -0.2305, -0.3307],
        [-1.5841,  0.1644,  1.4987,  ..., -0.8959, -0.2305, -0.3307],
        [-1

PackedSequence(data=tensor([[ 0.4043,  0.0725, -1.3084,  ...,  0.2704, -1.6287,  0.5179],
        [-0.4191,  1.8537, -0.9554,  ...,  0.2580, -2.0506,  1.5709],
        [ 0.8496, -0.5891,  0.6985,  ...,  0.2360, -1.5016, -0.8815],
        ...,
        [-1.5845,  0.1630,  1.4993,  ..., -0.8961, -0.2312, -0.3314],
        [-1.5845,  0.1630,  1.4993,  ..., -0.8961, -0.2312, -0.3314],
        [-1.5845,  0.1630,  1.4993,  ..., -0.8961, -0.2312, -0.3314]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 64, 58, 46, 35, 19, 10]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.8505, -0.5888,  0.6989,  ...,  0.2359, -1.5024, -0.8806],
        [-1.4784,  0.7732, -1.7460,  ..., -0.5081,  0.1024,  0.0909],
        [ 0.8505, -0.5888,  0.6989,  ...,  0.2359, -1.5024, -0.8806],
        ...,
        [-1.5846,  0.1630,  1.4993,  ..., -0.8961, -0.2312, -0.3314],
        [-1.5846,  0.1630,  1.4993,  ..., -0.8961, -0.2312, -0.3314],
        [-1

...

PackedSequence(data=tensor([[ 0.8990, -0.6570,  0.5741,  ...,  0.2938, -1.4307, -0.9060],
        [ 1.0559,  0.6285,  0.3062,  ...,  1.8861,  0.1321,  1.3481],
        [ 1.0559,  0.6285,  0.3062,  ...,  1.8861,  0.1321,  1.3481],
        ...,
        [-1.6869,  0.1925,  1.5044,  ..., -0.9563, -0.2265, -0.2169],
        [-1.6869,  0.1925,  1.5044,  ..., -0.9563, -0.2265, -0.2169],
        [-1.6869,  0.1925,  1.5044,  ..., -0.9563, -0.2265, -0.2169]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 63, 51, 38, 25, 15,  5]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[-0.2433,  0.1031, -0.1231,  ..., -0.4842, -0.6542,  1.5913],
        [ 0.8990, -0.6572,  0.5743,  ...,  0.2939, -1.4307, -0.9062],
        [ 0.8990, -0.6572,  0.5743,  ...,  0.2939, -1.4307, -0.9062],
        ...,
        [-1.6869,  0.1923,  1.5045,  ..., -0.9563, -0.2265, -0.2167],
        [-1.6869,  0.1923,  1.5045,  ..., -0.9563, -0.2265, -0.2167],
        [-1

PackedSequence(data=tensor([[ 0.8989, -0.6594,  0.5745,  ...,  0.2947, -1.4314, -0.9079],
        [ 0.0223, -0.6995,  0.3946,  ...,  0.1325, -0.5183, -0.4565],
        [ 0.8989, -0.6594,  0.5745,  ...,  0.2947, -1.4314, -0.9079],
        ...,
        [-1.6866,  0.1916,  1.5044,  ..., -0.9562, -0.2247, -0.2145],
        [-1.6866,  0.1916,  1.5044,  ..., -0.9562, -0.2247, -0.2145],
        [-1.6866,  0.1916,  1.5044,  ..., -0.9562, -0.2247, -0.2145]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 63, 51, 40, 23, 12,  6]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.8988, -0.6595,  0.5744,  ...,  0.2947, -1.4315, -0.9080],
        [ 0.8988, -0.6595,  0.5744,  ...,  0.2947, -1.4315, -0.9080],
        [ 0.8988, -0.6595,  0.5744,  ...,  0.2947, -1.4315, -0.9080],
        ...,
        [-1.6868,  0.1916,  1.5041,  ..., -0.9560, -0.2249, -0.2146],
        [-1.6868,  0.1916,  1.5041,  ..., -0.9560, -0.2249, -0.2146],
        [-1

PackedSequence(data=tensor([[ 0.9010, -0.6601,  0.5744,  ...,  0.2954, -1.4325, -0.9072],
        [ 0.0020, -0.7734,  0.1870,  ...,  1.1507, -0.6012,  0.9378],
        [ 0.9010, -0.6601,  0.5744,  ...,  0.2954, -1.4325, -0.9072],
        ...,
        [-1.6891,  0.1931,  1.5047,  ..., -0.9571, -0.2248, -0.2166],
        [-1.6891,  0.1931,  1.5047,  ..., -0.9571, -0.2248, -0.2166],
        [-1.6891,  0.1931,  1.5047,  ..., -0.9571, -0.2248, -0.2166]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 64, 55, 40, 27, 15,  2]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.9010, -0.6600,  0.5744,  ...,  0.2954, -1.4325, -0.9070],
        [ 0.9010, -0.6600,  0.5744,  ...,  0.2954, -1.4325, -0.9070],
        [ 0.8077, -1.1643, -1.8552,  ...,  0.1762, -0.2235, -1.2517],
        ...,
        [-1.6890,  0.1931,  1.5048,  ..., -0.9572, -0.2249, -0.2167],
        [-1.6890,  0.1931,  1.5048,  ..., -0.9572, -0.2249, -0.2167],
        [-1

PackedSequence(data=tensor([[ 0.9008, -0.6593,  0.5748,  ...,  0.2952, -1.4326, -0.9048],
        [ 0.8084, -1.1660, -1.8543,  ...,  0.1780, -0.2217, -1.2522],
        [ 1.0585,  0.6290,  0.3009,  ...,  1.8911,  0.1334,  1.3480],
        ...,
        [-1.6879,  0.1918,  1.5050,  ..., -0.9560, -0.2242, -0.2150],
        [-1.6879,  0.1918,  1.5050,  ..., -0.9560, -0.2242, -0.2150],
        [-1.6879,  0.1918,  1.5050,  ..., -0.9560, -0.2242, -0.2150]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 60, 52, 42, 25, 14,  6]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.9008, -0.6593,  0.5750,  ...,  0.2953, -1.4325, -0.9047],
        [-0.2438,  0.1067, -0.1211,  ..., -0.4866, -0.6529,  1.5966],
        [ 0.8086, -1.1660, -1.8541,  ...,  0.1780, -0.2217, -1.2522],
        ...,
        [-1.6877,  0.1917,  1.5052,  ..., -0.9560, -0.2239, -0.2147],
        [-1.6877,  0.1917,  1.5052,  ..., -0.9560, -0.2239, -0.2147],
        [-1

PackedSequence(data=tensor([[ 0.8991, -0.6601,  0.5759,  ...,  0.2957, -1.4322, -0.9034],
        [-0.6449,  0.6366, -0.4804,  ...,  0.0581,  0.0429, -0.2882],
        [ 0.8991, -0.6601,  0.5759,  ...,  0.2957, -1.4322, -0.9034],
        ...,
        [-1.6874,  0.1922,  1.5052,  ..., -0.9561, -0.2239, -0.2160],
        [-1.6874,  0.1922,  1.5052,  ..., -0.9561, -0.2239, -0.2160],
        [-1.6874,  0.1922,  1.5052,  ..., -0.9561, -0.2239, -0.2160]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 63, 51, 40, 30, 19,  7]), sorted_indices=None, unsorted_indices=None)
PackedSequence(data=tensor([[ 0.8990, -0.6601,  0.5758,  ...,  0.2958, -1.4322, -0.9033],
        [ 0.8990, -0.6601,  0.5758,  ...,  0.2958, -1.4322, -0.9033],
        [ 0.8990, -0.6601,  0.5758,  ...,  0.2958, -1.4322, -0.9033],
        ...,
        [-1.6875,  0.1920,  1.5050,  ..., -0.9561, -0.2240, -0.2161],
        [-1.6875,  0.1920,  1.5050,  ..., -0.9561, -0.2240, -0.2161],
        [-1

In [24]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    tools.save_model(encoder, '5_encoder.pth')
    tools.save_model(decoder, '5_decoder.pth')
else:
    hidden_size = 256
    encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size)
    tools.load_model(encoder, '5_encoder.pth', device)
    
    decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size)
    tools.load_model(decoder, '5_decoder.pth', device)

Do you want to save the model (type yes to confirm)? yes
Model saved to 5_encoder.pth.
Do you want to save the model (type yes to confirm)? yes
Model saved to 5_decoder.pth.


In [25]:
# This cell tests training accuracy

## Evaluation

Next we need to implement a function that converts a source sequence to an output sequence using the trained sequence-to-sequence model.

In [29]:
def translate(encoder, decoder, src_seq):
    """Translate given sentence src_seq using trained encoder and decoder.
    
    Args:
      encoder (Encoder): Trained encoder.
      decoder (Decoder): Trained decoder.
      src_seq of shape (src_seq_length,): LongTensor of word indices of the source sequence.
    
    Returns:
      out_seq of shape (out_seq_length,): LongTensor of word indices of the output sequence.
    """
    # YOUR CODE HERE
    try:
        hidden = encoder.init_hidden().to(device)
        src_seq = src_seq.view(-1, 1).to(device)
        out , hidden = encoder(src_seq,[src_seq.size(0)] , hidden)
        out , hidden = decoder(hidden)
        decoder_outputs = torch.max(out,dim=2)[1]
        return decoder_outputs
    except:
        raise NotImplementedError()

In [30]:
def test_translate_shapes():
    src_seq = torch.tensor([1, 2, 3, 4]).to(device)
    out_seq = translate(encoder, decoder, src_seq)
    assert out_seq.shape[0] <= MAX_LENGTH, \
        f"Too long output sequence: tgt_seq.shape[0]={tgt_seq.shape[0]}"
    print('Success')

test_translate_shapes()

PackedSequence(data=tensor([[-1.6890,  0.1918,  1.5031,  ..., -0.9552, -0.2277, -0.2183],
        [-0.2122,  1.7797, -0.9777,  ...,  0.3391, -2.2032,  1.5433],
        [ 1.5658,  1.0050, -0.6939,  ...,  0.4823,  0.1171,  0.9921],
        [-0.5672, -0.6882,  0.2482,  ...,  0.9172, -1.3292,  2.3397]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([1, 1, 1, 1]), sorted_indices=None, unsorted_indices=None)
Success


Let us now translate random sentences from the training set and print the source, target, and produced output.

If you trained the model well enough, the model should memorize the training data well.

In [31]:
# Translate random sentences from the training set
print('Translate training data:')
print('-----------------------------')
for i in range(5):
    src_sentence, tgt_sentence = trainset[np.random.choice(len(trainset))]
    print('SRC:', ' '.join(trainset.input_lang.index2word[i.item()] for i in src_sentence))
    print('TGT:', ' '.join(trainset.output_lang.index2word[i.item()] for i in tgt_sentence))
    out_sentence = translate(encoder, decoder, src_sentence)
    print('OUT:', ' '.join(trainset.output_lang.index2word[i.item()] for i in out_sentence))
    print('')

Translate training data:
-----------------------------
SRC: tu es si belle dans cette robe ! EOS
TGT: you re so beautiful in that dress . EOS
PackedSequence(data=tensor([[ 0.3936,  0.0738, -1.4573,  ...,  0.3330, -1.5065,  0.5300],
        [-0.7665,  1.1029, -0.3305,  ..., -1.2278, -1.0094, -0.6278],
        [-0.1730,  0.4422,  0.2513,  ...,  1.6282,  0.0563, -0.8459],
        ...,
        [ 1.6057, -0.0306, -0.1469,  ..., -1.6759,  0.0730,  0.3636],
        [ 1.0496,  0.1356,  0.3527,  ...,  0.0754,  0.2407, -1.6080],
        [-1.6890,  0.1918,  1.5031,  ..., -0.9552, -0.2277, -0.2183]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([1, 1, 1, 1, 1, 1, 1, 1, 1]), sorted_indices=None, unsorted_indices=None)
OUT: you re so beautiful in that dress . EOS EOS

SRC: vous etes des cretins . EOS
TGT: you are morons . EOS
PackedSequence(data=tensor([[ 0.5080,  1.2527, -1.1351,  ...,  1.2083,  2.6243,  0.7065],
        [-1.2052,  0.1859, -2.4632,  ...,  0.6781,  0.9663, -0.238

Now we translate random sentences from the test set. A well-trained model should output sentences that look similar to the target ones. The mistakes are usually done for words that were rare in the training set.

In [32]:
testset = TranslationDataset(data_dir, train=False)

In [33]:
print('Translate test data:')
print('-----------------------------')
for i in range(5):
    src_sentence, tgt_sentence = testset[np.random.choice(len(testset))]
    print('SRC:', ' '.join(testset.input_lang.index2word[i.item()] for i in src_sentence))
    print('TGT:', ' '.join(testset.output_lang.index2word[i.item()] for i in tgt_sentence))
    out_sentence = translate(encoder, decoder, src_sentence)
    print('OUT:', ' '.join(testset.output_lang.index2word[i.item()] for i in out_sentence))
    print('')

Translate test data:
-----------------------------
SRC: je ne suis pas occupe . EOS
TGT: i m not busy . EOS
PackedSequence(data=tensor([[ 0.8991, -0.6610,  0.5752,  ...,  0.2959, -1.4321, -0.9027],
        [-0.0271,  0.7982,  1.6409,  ..., -1.0739,  0.6021, -0.2495],
        [ 2.0962,  0.3000, -0.4227,  ..., -0.5595, -1.4396, -0.6483],
        ...,
        [-1.7943, -1.3819,  0.9851,  ...,  0.1078, -0.1593,  0.2307],
        [-1.9663,  1.8077,  0.2535,  ..., -0.7386,  0.7112,  0.1785],
        [-1.6890,  0.1918,  1.5031,  ..., -0.9552, -0.2277, -0.2183]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([1, 1, 1, 1, 1, 1, 1]), sorted_indices=None, unsorted_indices=None)
OUT: i m not busy busy . EOS EOS EOS EOS

SRC: il est etudiant . EOS
TGT: he is a student . EOS
PackedSequence(data=tensor([[ 0.8107, -1.1672, -1.8538,  ...,  0.1790, -0.2224, -1.2545],
        [ 0.1443,  0.0887,  1.3498,  ...,  0.4902,  1.9784, -0.7329],
        [ 1.0223, -0.0500,  0.3705,  ...,  0.9585